In [1]:
!pip install -r ../requirements.txt

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
##Load API key ------------------
f = open('../apikeys/api.txt', 'r')
key = f.readline()
os.environ['OPENAI_API_KEY'] = key

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.1, max_tokens=512, model="text-davinci-003")

In [4]:
sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


In [5]:
#######
# The following code implement a simple rule based checker for 
# a specific 4x4 sudoku puzzle.
#######

from typing import Tuple
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity
import re

In [6]:
class MyChecker(ToTChecker):
    def evaluate(self,
        problem_description: str,
        thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID


In [7]:
#######
# Testing the MyChecker class above:
#######

checker = MyChecker()
assert checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",)) == ThoughtValidity.VALID_FINAL
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",)) == ThoughtValidity.VALID_INTERMEDIATE
assert checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",)) == ThoughtValidity.INVALID


In [11]:
#######
# Initialize and run the ToT chain, 
# with maximum number of interactions k set to 30 and 
# the maximum number child thoughts c set to 8.
#######

from langchain_experimental.tot.base import ToTChain

tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=10, c=5, verbose=True, verbose_llm=False)
solution = tot_chain.run(problem_description=problem_description)



> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,1,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,*
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,1
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,4,1
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,4,2,1
            Thought: 3,*,*,2|1,4,3,*|*,1,*,3|4,3,2,1

> Finished chain.


In [16]:
print(solution, solution == "")

No solution found True


In [18]:
for i in range(1, 11, 1):
    k = i*5
    tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=k, c=5, verbose=True, verbose_llm=False)
    solution = tot_chain.run(problem_description=problem_description)
    if solution != "No solution found":
        print("solution found at k =", k)
        print(solution)
        break



> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,1,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,*

> Finished chain.


> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,*,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,2,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,2,1,*
    Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,*
        Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,1
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,2,4
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,3,4,1
            Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,4,2,1
            Thought: 3,4,*,2|1,*,3,*|*,1,*,3|4,3,2,1

> Finished chain.


> Entering new ToTChain chain...
Starting the ToT solve procedure.
Thought: 3,*,*,2|1,*,3,*|*,1,*,3|4,1,*,*
Thought: 3,*,*,2|1,

35